<p style="padding: 10px;
          color:black;
          text-align: center;
          font-size:250%;">
PySpark Tutorial
</p>

# What is Spark?
- Spark is one of the latest technologies being used to quickly and easily handle Big Data
- It is an open source project on **Apache**
- It was first released in February 2013 and has exploded in popularity due to it’s ease of use and speed
- It was created at the AMPLab at UC Berkeley
- Spark is 100 times faster than **Hadoop MapReduce**
- Spark does not store anything unless any action is applied on the data

![image.png](attachment:679c1237-df7b-44cd-8a55-32ba9cbd0e3e.png)

# Libraries and Utilities

We need to install pyspark first

In [1]:
!pip install pyspark

In [1]:
import os
import warnings
warnings.filterwarnings('ignore')
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, FloatType
from pyspark.sql.functions import split, count, when, isnan, col, regexp_replace
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Creating a SparkSession

In [1]:
spark = SparkSession.builder.appName('First Session').getOrCreate()

print('Spark Version: {}'.format(spark.version))

# Loading Data

In [1]:
#Defining a Schema
schema = StructType([StructField('mpg', FloatType(), nullable = True),
                     StructField('cylinders', IntegerType(), nullable = True),
                     StructField('displacement', FloatType(), nullable = True),
                     StructField('horsepower', StringType(), nullable = True),
                     StructField('weight', IntegerType(), nullable = True),
                     StructField('acceleration', FloatType(), nullable = True),
                     StructField('model year', IntegerType(), nullable = True),
                     StructField('origin', IntegerType(), nullable = True),
                     StructField('car name', StringType(), nullable = True)])

file_path = '/kaggle/input/autompg-dataset/auto-mpg.csv'

df = spark.read.csv(file_path,
                    header = True,
                    inferSchema = True,
                    nanValue = '?')

df.show(5)

In [1]:
#Check Missing Values
def check_missing(dataframe):
    
    return dataframe.select([count(when(isnan(c) | col(c).isNull(), c)). \
                             alias(c) for c in dataframe.columns]).show()

check_missing(df)

In [1]:
#Handling Missing Values
df = df.na.drop()

df = df.withColumn("horsepower", df["horsepower"].cast(IntegerType())) #convert horsepower from string to int

df.show(5)

# PySpark DataFrame Basics

- Spark DataFrames hold data in a column and row format.
- Each column represents some feature or variable.
- Each row represents an individual data point.
- They are able to input and output data from a wide variety of sources.
- We can then use these DataFrames to apply various transformations on the data.
- At the end of the transformation calls, we can either show or collect the results to display or for some final processing.

In [1]:
#Check column names
df.columns

In [1]:
#Display data with pandas format
df.toPandas().head()

In [1]:
#Check the schema
df.printSchema()

In [1]:
#Renaming Columns
df = df.withColumnRenamed('model year', 'model_year')

df = df.withColumnRenamed('car name', 'car_name')

df.show(3)

In [1]:
#Get infos from first 4 rows
for car in df.head(4):
    print(car, '\n')

In [1]:
#statistical summary of dataframe
df.describe().show()

- <code>describe()</code> represents the statiscal summary of dataframe but it also uses the string variables

In [1]:
#describe with specific variables
df.describe(['mpg', 'horsepower']).show()

In [1]:
#describe with numerical columns
def get_num_cols(dataframe):
    
    num_cols = [col for col in dataframe.columns if dataframe.select(col). \
                dtypes[0][1] in ['double', 'int']]
    
    return num_cols

num_cols = get_num_cols(df)
    
df.describe(num_cols).show()

# Spark DataFrame Basic Operations

## Filtering & Sorting

In [1]:
#Lets get the cars with mpg more than 23
df.filter(df['mpg'] > 23).show(5)

In [1]:
#Multiple Conditions
df.filter((df['horsepower'] > 80) & 
          (df['weight'] > 2000)).select('car_name').show(5)

In [1]:
#Sorting
df.filter((df['mpg'] > 25) & (df['origin'] == 2)). \
orderBy('mpg', ascending = False).show(5)

In [1]:
#Get the cars with 'volkswagen' in their names, and sort them by model year and horsepower
df.filter(df['car_name'].contains('volkswagen')). \
orderBy(['model_year', 'horsepower'], ascending=[False, False]).show(5)

In [1]:
df.filter(df['car_name'].like('%volkswagen%')).show(3)

## Filtering with SQL

In [1]:
#Get the cars with 'toyota' in their names
df.filter("car_name like '%toyota%'").show(5)

In [1]:
df.filter('mpg > 22').show(5)

In [1]:
#Multiple Conditions
df.filter('mpg > 22 and acceleration < 15').show(5)

In [1]:
df.filter('horsepower == 88 and weight between 2600 and 3000') \
.select(['horsepower', 'weight', 'car_name']).show()

# GroupBy and Aggregate Operations

In [1]:
#Brands
df.createOrReplaceTempView('auto_mpg')

df = df.withColumn('brand', split(df['car_name'], ' ').getItem(0)).drop('car_name')

#Replacing Misspelled Brands
auto_misspelled = {'chevroelt': 'chevrolet',
                   'chevy': 'chevrolet',
                   'vokswagen': 'volkswagen',
                   'vw': 'volkswagen',
                   'hi': 'harvester',
                   'maxda': 'mazda',
                   'toyouta': 'toyota',
                   'mercedes-benz': 'mercedes'}

for key in auto_misspelled.keys():
    
    df = df.withColumn('brand', regexp_replace('brand', key, auto_misspelled[key]))

df.show(5)

In [1]:
#Avg Acceleration by car brands
df.groupBy('brand').agg({'acceleration': 'mean'}).show(5)

In [1]:
#Max MPG by car brands
df.groupBy('brand').agg({'mpg': 'max'}).show(5)

# Machine Learning

- Machine learning is a method of data analysis that automates analytical model building. 
- Using algorithms that iteratively learn from data, machine learning allows computers to find hidden insights without being explicitly programmed where to look.

## Supervised Learning

- Spark’s MLlib is mainly designed for **Supervised** and **Unsupervised Learning** tasks, with most of its algorithms falling under those two categories.
- Supervised learning algorithms are trained using labeled examples, such as an input where the desired output is known. 
- For example, a piece of equipment could have data points labeled either “F” (failed) or “R” (runs). 
- The learning algorithm receives a set of inputs along with the corresponding correct outputs, and the algorithm learns by comparing its actual output with correct outputs to find errors. 
- It then modifies the model accordingly.
- Through methods like classification, regression, prediction and gradient boosting, supervised learning uses patterns to predict the values of the label on additional unlabeled data. 
- Supervised learning is commonly used in applications where historical data predicts likely future events.
- For example, it can anticipate when credit card transactions are likely to be fraudulent or which insurance customer is likely to file a claim.
- Or it can attempt to predict the price of a house based on different features for houses for which we have historical price data.

## Unsupervised Learning

- Unsupervised learning is used against data that has no historical labels. 
- The system is not told the "right answer." The algorithm must figure out what is being shown. 
- The goal is to explore the data and find some structure within.
- For example, it can find the main attributes that separate customer segments from each other. 
- Popular techniques include self-organizing maps, nearest-neighbor mapping, k-means clustering and singular value decomposition.
- One issue is that it can be difficult to evaluate results of an unsupervised model!

# Machine Learning with PySpark

- Spark has its own MLlib for Machine Learning.
- The future of MLlib utilizes the Spark 2.0 DataFrame syntax.
- One of the main “quirks” of using MLlib is that you need to format your data so that eventually  it just has one or two columns:
    - Features, Labels (Supervised)
    - Features (Unsupervised)
- This requires a little more data processing work than some other machine learning libraries, but the big upside is that this exact same syntax works with distributed data, which is no small feat for what is going on “under the hood”!
- When working with Python and Spark with MLlib, the documentation examples are always with nicely formatted data.
- A huge part of learning MLlib is getting comfortable with the documentation!
- Being able to master the skill of finding information (not memorization) is the key to becoming a great Spark and Python developer!
- Let’s jump to it now!

Learn more in here: https://spark.apache.org/mllib/

![image.png](attachment:9f379daa-23dc-4f1d-9886-47f6a23764a7.png)

# Preprocessing

## Encoding Brands

In [1]:
#Check brand frequences first
df.groupby('brand').count().orderBy('count', ascending = False).show(5)

In [1]:
def one_hot_encoder(dataframe, col):
    
    indexed = StringIndexer().setInputCol(col).setOutputCol(col + '_cat'). \
    fit(dataframe).transform(dataframe) #converting categorical values into category indices
    
    ohe = OneHotEncoder().setInputCol(col + '_cat').setOutputCol(col + '_OneHotEncoded'). \
    fit(indexed).transform(indexed)
    
    ohe = ohe.drop(*[col, col + '_cat'])
    
    return ohe

df = one_hot_encoder(df, col = 'brand')
df.show(5)

In [1]:
#Vector Assembler
def vector_assembler(dataframe, indep_cols):
    
    assembler = VectorAssembler(inputCols = indep_cols,
                                outputCol = 'features')

    output = assembler.transform(dataframe).drop(*indep_cols)
    
    return output

df = vector_assembler(df, indep_cols = df.drop('mpg').columns)
df.show(5)

## Train-Test Split

In [1]:
train_data, test_data = df.randomSplit([0.8, 0.2])

print('Train Shape: ({}, {})'.format(train_data.count(), len(train_data.columns)))
print('Test Shape: ({}, {})'.format(test_data.count(), len(test_data.columns)))

# Multiple Linear Regression with PySpark

## Fit the Model

In [1]:
lr = LinearRegression(labelCol = 'mpg',
                      featuresCol = 'features',
                      regParam = 0.3) #avoid overfitting

lr = lr.fit(train_data)

## Model Evaluation

In [1]:
def evaluate_reg_model(model, test_data):
    
    print(model.__class__.__name__.center(70, '-'))
    model_results = model.evaluate(test_data)
    print('R2: {}'.format(model_results.r2))
    print('MSE: {}'.format(model_results.meanSquaredError))
    print('RMSE: {}'.format(model_results.rootMeanSquaredError))
    print('MAE: {}'.format(model_results.meanAbsoluteError))
    print(70*'-')

evaluate_reg_model(lr, test_data)

In [1]:
#End Session
spark.stop()

**If you liked this notebook, please upvote** 😊

**If you have any suggestions or questions, feel free to comment!**

**Best Wishes!**